In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
X=pd.read_csv("../input/dmmlassignment2/x_train.csv")
y=pd.read_csv("../input/dmml-assignment/y_train.csv")
actual_dataset = pd.read_csv("../input/dmml-assignment/x_test.csv")
#y_test = pd.read_csv("../input/dmmlassignment2/y_test_baseline.csv")

In [ ]:
X = X[X.columns[1:]]
X.columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3', 'Wilderness_Area_4', 
'Soil_Type_1', 'Soil_Type_2','Soil_Type_3','Soil_Type_4','Soil_Type_5','Soil_Type_6','Soil_Type_7','Soil_Type_8','Soil_Type_9','Soil_Type_10',
'Soil_Type_11','Soil_Type_12','Soil_Type_13','Soil_Type_14','Soil_Type_15','Soil_Type_16','Soil_Type_17','Soil_Type_18','Soil_Type_19','Soil_Type_20',
'Soil_Type_21','Soil_Type_22','Soil_Type_23','Soil_Type_24','Soil_Type_25','Soil_Type_26','Soil_Type_27','Soil_Type_28','Soil_Type_29','Soil_Type_30',
'Soil_Type_31','Soil_Type_32','Soil_Type_33','Soil_Type_34','Soil_Type_35','Soil_Type_36','Soil_Type_37','Soil_Type_38','Soil_Type_39','Soil_Type_40']

actual_dataset = actual_dataset[actual_dataset.columns[1:]]
actual_dataset.columns = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area_1', 'Wilderness_Area_2', 'Wilderness_Area_3', 'Wilderness_Area_4', 
'Soil_Type_1', 'Soil_Type_2','Soil_Type_3','Soil_Type_4','Soil_Type_5','Soil_Type_6','Soil_Type_7','Soil_Type_8','Soil_Type_9','Soil_Type_10',
'Soil_Type_11','Soil_Type_12','Soil_Type_13','Soil_Type_14','Soil_Type_15','Soil_Type_16','Soil_Type_17','Soil_Type_18','Soil_Type_19','Soil_Type_20',
'Soil_Type_21','Soil_Type_22','Soil_Type_23','Soil_Type_24','Soil_Type_25','Soil_Type_26','Soil_Type_27','Soil_Type_28','Soil_Type_29','Soil_Type_30',
'Soil_Type_31','Soil_Type_32','Soil_Type_33','Soil_Type_34','Soil_Type_35','Soil_Type_36','Soil_Type_37','Soil_Type_38','Soil_Type_39','Soil_Type_40']

y = y[y.columns[1:]]
y.columns=['Cover_Type']

# Data cleaning
rem = []
for c in X.columns:
    if X[c].std() == 0: #standard deviation is zero
        rem.append(c)
X.drop(rem,axis=1,inplace=True)

In [ ]:
dataset = pd.concat([X, y], axis=1)
dataset.groupby('Cover_Type').size()

In [ ]:
import seaborn as sns
sns.distplot(dataset['Horizontal_Distance_To_Hydrology']);

In [ ]:
#correlation matrix
corrmat = dataset.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
max_size = 198310

lst = [dataset]
for class_index, group in dataset.groupby('Cover_Type'):
    lst.append(group.sample(max_size-len(group), replace=True))
dataset = pd.concat(lst)

y = dataset.iloc[:, -1:]
X = dataset.iloc[:,:-1] 

In [ ]:
# Removing rows with missing value

from numpy import nan

result = pd.concat([X, y], axis=1)
print(result.shape) # number of matrix before removing rows with zero values
result[["Horizontal_Distance_To_Fire_Points"]] = result[["Horizontal_Distance_To_Fire_Points"]].replace(0, nan)
result.dropna(inplace=True)
#result.fillna(result[["Horizontal_Distance_To_Fire_Points"]].mean(), inplace=True)

result[["Aspect"]] = result[["Aspect"]].replace(0, nan)
result.dropna(inplace=True)
#result.fillna(result[["Aspect"]].mean(), inplace=True)

result.fillna(result.mean(), inplace=True)


y = result.iloc[:, -1:]
X = result.iloc[:,:-1] 
print(y.shape)
print(X.shape)

In [ ]:
# Normalization between highly correlated attributes
result = pd.concat([X["Hillshade_9am"],X["Hillshade_3pm"]], axis=1)
result=((result-result.min())/(result.max()-result.min()))*2
X["Hillshade_9am"] = result.iloc[:, :-1]
X["Hillshade_3pm"] = result.iloc[:, -1]

result = pd.concat([X['Horizontal_Distance_To_Hydrology'],X['Vertical_Distance_To_Hydrology']], axis=1)
result=((result-result.min())/(result.max()-result.min()))*2
X['Horizontal_Distance_To_Hydrology'] = result.iloc[:, :-1]
X['Vertical_Distance_To_Hydrology'] = result.iloc[:, -1]

result = pd.concat([X['Slope'],X['Hillshade_Noon']], axis=1)
result=((result-result.min())/(result.max()-result.min()))*2
X['Slope'] = result.iloc[:, :-1]
X['Hillshade_Noon'] = result.iloc[:, -1]


In [ ]:
X.shape

# Normalization
X=((X-X.min())/(X.max()-X.min()))*2


In [ ]:
import seaborn as sns
sns.boxplot(x=X['Slope'])

# Removing outliers from Slope
median_X = X['Slope'].median()
X['Slope'] = np.where(X['Slope'] < median_X, median_X, X['Slope'])

# Removing outliers from Horizontal_Distance_To_Hydrology
median_H = X['Horizontal_Distance_To_Hydrology'].median()
X['Horizontal_Distance_To_Hydrology'] = np.where(X['Horizontal_Distance_To_Hydrology'] < median_H, median_H, X['Horizontal_Distance_To_Hydrology'])

# Removing outliers from Vertical_Distance_To_Hydrology
median_V = X['Vertical_Distance_To_Hydrology'].median()
X['Vertical_Distance_To_Hydrology'] = np.where(X['Vertical_Distance_To_Hydrology'] < median_V, median_V, X['Vertical_Distance_To_Hydrology'])

# Removing outliers from 'Horizontal_Distance_To_Roadways'
median_V = X['Horizontal_Distance_To_Roadways'].median()
X['Horizontal_Distance_To_Roadways'] = np.where(X['Horizontal_Distance_To_Roadways'] < median_V, median_V, X['Horizontal_Distance_To_Roadways'])

# Removing outliers from 'Hillshade_9am'
median_V = X['Hillshade_9am'].median()
X['Hillshade_9am'] = np.where(X['Hillshade_9am'] < median_V, median_V, X['Hillshade_9am'])

In [ ]:
# Removing outliers from y_train
median = y.loc[y['Cover_Type']<3, 'Cover_Type'].median()
y['Cover_Type'] = np.where(y['Cover_Type'] > median, median, y['Cover_Type'])

In [ ]:
import seaborn as sns
sns.distplot(y['Cover_Type']);

In [ ]:
median

In [ ]:
y.head(20)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10, stratify=y['Cover_Type'])
# test_size = 0.30, random state = 10
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
# By far best result
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

#Base estimator
base_estimator = DecisionTreeClassifier(random_state=10,max_depth=13)

n_list = [100]

for n_estimators in n_list:
    #Set the base model
    model = BaggingClassifier(n_jobs=-1,base_estimator=base_estimator, n_estimators=n_estimators, random_state=10)

forest = BaggingClassifier(n_estimators=20) # initially 10
forest.fit(x_train, y_train.values.ravel())

In [ ]:
# Accuracy score
from sklearn.metrics import accuracy_score

y_pred = forest.predict(actual_dataset)

#y_pred = forest.predict(x_test)
#print(accuracy_score(y_test, y_pred))

In [ ]:

index = list(range(len(actual_dataset)))
y_pred = y_pred.astype('int')

df = pd.DataFrame(data=np.vstack((index,y_pred)).T, index=index, columns=["id","Cover_Type"])
df.to_csv('NORM_result_2.csv',index=False)

#df.to_csv('mycsvfile_5.csv',index=False)

In [ ]:
y_pred

In [ ]:
y.head()